<a href="https://colab.research.google.com/github/GinuraAdikari/InsightHive/blob/Sentiment_Analysis/Model_ABSA_DataSet2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***Import Required Libraries***

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer, AdamW, get_scheduler
import numpy as np
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report

In [2]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [3]:
ABSA = pd.read_csv("ABSA_dataset.csv")

In [4]:
ABSA.head()

,tokens,aspect_mask,sentiment
0,"['one', 'best', 'game', 'music', 'soundtrack',...","[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, ...",1
1,"['best', 'purchase', 'ever', 'bought', 'ex', '...","[1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, ...",1
2,"['book', 'slow', 'weak', 'one', 'best', '##st'...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, ...",0
3,"['must', '##rea', '##d', 'every', 'southern', ...","[0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, ...",1
4,"['horrible', 'watch', 'napoleon', 'want', 'fun...","[1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",0


🔹 Understanding Your Dataset Format. Your dataset consists of:


---



✅ "tokens" → List of words in each sentence (already tokenized).

✅ "aspect_mask" → Binary mask (1 = aspect-related, 0 = others).

✅ "sentiment" → Label (0 = Negative, 1 = Positive)

#***Train & Test Split***

In [5]:
from sklearn.model_selection import train_test_split

# train (80%) and test (20%)
train_df, test_df = train_test_split(ABSA, test_size=0.2, random_state=42, stratify=ABSA["sentiment"])

print("Train and test datasets created!")

Train and test datasets created!


In [6]:
train_df.head()

,tokens,aspect_mask,sentiment
163,"['hey', 'hey', 'wanted', 'say', 'awesome', 'cd...","[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...",1
549,"['fantastic', 'information', 'i', 've', 'learn...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
988,"['son', 'law', 'loved']","[1, 1, 0]",1
512,"['seen', 'old', 'movie', 'many', 'time', 'grea...","[0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0]",1
882,"['work', 'great', 'house', 'extend', 'wi', '##...","[0, 1, 1, 1, 1, 1, 1, 1]",1


In [7]:
test_df.head()

,tokens,aspect_mask,sentiment
684,"['book', 'must', 'must', 'pas', 'child', 'fina...","[1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, ...",1
972,"['reading', 'review', 'almost', 'regretted', '...","[1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...",1
1045,"['bought', 'husband', 'galaxy', 'tab', 'use', ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]",1
845,"['fast', 'easy', 'setup', 'ca', 'nt', 'ask', '...","[1, 1, 1, 0, 0, 0, 0, 0]",1
936,"['log', '##ite', '##ch', 'wireless', 'gaming',...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, ...",1


In [8]:
print(train_df["sentiment"].value_counts())

sentiment
1    718
0    330
Name: count, dtype: int64


#***Aspect-Based DataSet Class***

In [9]:
import torch
from torch.utils.data import Dataset

class ABSADataset(Dataset):
    def __init__(self, df, tokenizer, max_length=128):
        self.tokens = df["tokens"].tolist()  # ✅ Use pre-tokenized text
        self.aspect_masks = df["aspect_mask"].tolist()  # ✅ Aspect mask is already available
        self.sentiments = df["sentiment"].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        tokens = self.tokens[idx]  # ✅ Already tokenized

        if isinstance(tokens, str):
          tokens = eval(tokens)  # ✅ Convert string representation of list to actual list

        aspect_mask = self.aspect_masks[idx]  # ✅ Already provided
        if isinstance(aspect_mask, str):
          aspect_mask = eval(aspect_mask)  # ✅ Convert if stored as a string

        sentiment = self.sentiments[idx]

        # ✅ Convert tokens to IDs
        input_ids = self.tokenizer.convert_tokens_to_ids(tokens)

        # ✅ Padding
        padding_length = self.max_length - len(input_ids)
        if padding_length > 0:
            input_ids += [0] * padding_length
            aspect_mask += [0] * padding_length  # ✅ Pad aspect mask too
        else:
            input_ids = input_ids[:self.max_length]
            aspect_mask = aspect_mask[:self.max_length]

        return {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "attention_mask": torch.tensor([1] * len(input_ids), dtype=torch.long),
            "aspect_mask": torch.tensor(aspect_mask, dtype=torch.long),
            "labels": torch.tensor(sentiment, dtype=torch.long)
        }

In [ ]:
class ABSADataset(Dataset):
    def __init__(self, df, tokenizer, max_length=128):
        self.tokens = df["tokens"].tolist()
        self.aspect_masks = df["aspect_mask"].tolist()
        self.sentiments = df["sentiment"].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        tokens = self.tokens[idx]

        # ✅ Ensure tokens are lists (avoid string format issues)
        if isinstance(tokens, str):
            tokens = eval(tokens)
        elif not isinstance(tokens, list):
            tokens = [tokens]

        input_ids = self.tokenizer.convert_tokens_to_ids(tokens)

        # ✅ Use "aspect_mask" directly
        aspect_mask = self.aspect_masks[idx]
        if isinstance(aspect_mask, str):
            aspect_mask = eval(aspect_mask)
        elif not isinstance(aspect_mask, list):
            aspect_mask = [aspect_mask]

        # ✅ Create attention mask (1 for actual tokens, 0 for padding)
        attention_mask = [1] * len(input_ids)

        # ✅ Padding & Truncation
        padding_length = self.max_length - len(input_ids)
        if padding_length > 0:
            input_ids += [0] * padding_length
            attention_mask += [0] * padding_length
            aspect_mask += [0] * padding_length
        else:
            input_ids = input_ids[:self.max_length]
            attention_mask = attention_mask[:self.max_length]
            aspect_mask = aspect_mask[:self.max_length]

        return {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
            "aspect_mask": torch.tensor(aspect_mask, dtype=torch.long),
            "labels": torch.tensor(self.sentiments[idx], dtype=torch.long)
        }

print("✅ ABSADataset class loaded successfully!")


#***Aspect-Aware BERT Model***

In [10]:
class BertForABSA(nn.Module):
    def __init__(self, bert_model="bert-base-uncased", num_labels=2):
        super(BertForABSA, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model)
        self.aspect_attention = nn.Linear(self.bert.config.hidden_size, 1)  # Aspect-aware attention
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)  # Sentiment classification
        self.dropout = nn.Dropout(0.1)

    def forward(self, input_ids, attention_mask, aspect_mask, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state

        # Apply aspect-aware attention
        aspect_weights = torch.softmax(self.aspect_attention(hidden_states), dim=1)
        aspect_weights = aspect_weights * aspect_mask.unsqueeze(-1)
        aspect_representations = hidden_states * aspect_weights

        # Use sum instead of mean for better aspect representation
        pooled_output = torch.sum(aspect_representations, dim=1)

        # Classification layer
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss = loss_fn(logits, labels)

        return loss, logits

In [ ]:
class BertForABSA(nn.Module):
    def __init__(self, bert_model="bert-base-uncased", num_labels=2):
        super(BertForABSA, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model)
        self.aspect_attention = nn.Linear(self.bert.config.hidden_size, 1)  # Aspect-aware attention
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)  # Sentiment classification
        self.dropout = nn.Dropout(0.1)

    def forward(self, input_ids, attention_mask, aspect_mask, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state

        # ✅ Apply aspect-aware attention
        aspect_weights = torch.softmax(self.aspect_attention(hidden_states), dim=1)
        aspect_weights = aspect_weights * aspect_mask.unsqueeze(-1)
        aspect_representations = hidden_states * aspect_weights

        # ✅ Use sum instead of mean for better aspect representation
        pooled_output = torch.sum(aspect_representations, dim=1)

        # Classification layer
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss = loss_fn(logits, labels)

        return loss, logits


#***Training Pipeline***

In [11]:
# Initialize device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Load dataset
train_dataset = ABSADataset(train_df, tokenizer)
test_dataset = ABSADataset(test_df, tokenizer)

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [12]:
for batch in train_dataloader:
    input_ids = batch["input_ids"][0].tolist()
    aspect_mask = batch["aspect_mask"][0].tolist()
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    print("\n📌 Tokenized Sentence with Aspect Mask:\n")
    for token, mask in zip(tokens, aspect_mask):
        mark = "[ASPECT]" if mask == 1 else ""
        print(f"{token:<10} {mark}")  # ✅ Ensures aspect words are correctly marked

    break


📌 Tokenized Sentence with Aspect Mask:

ipad       [ASPECT]
mini       [ASPECT]
absolutely [ASPECT]
perfect    [ASPECT]
protection [ASPECT]
function   [ASPECT]
style      [ASPECT]
graphic    [ASPECT]
design     [ASPECT]
cover      [ASPECT]
class      [ASPECT]
##y        [ASPECT]
receives   
positive   [ASPECT]
reaction   [ASPECT]
everyone   [ASPECT]
seen       
ipad       
fit        
perfectly  
stay       
secure     
cut        
##out      
right      [ASPECT]
place      [ASPECT]
definitely 
great      
choice     
anyone     [ASPECT]
want       
class      [ASPECT]
##y        [ASPECT]
functional [ASPECT]
well       [ASPECT]
##made     [ASPECT]
ipad       [ASPECT]
cover      [ASPECT]
[PAD]      
[PAD]      
[PAD]      
[PAD]      
[PAD]      
[PAD]      
[PAD]      
[PAD]      
[PAD]      
[PAD]      
[PAD]      
[PAD]      
[PAD]      
[PAD]      
[PAD]      
[PAD]      
[PAD]      
[PAD]      
[PAD]      
[PAD]      
[PAD]      
[PAD]      
[PAD]      
[PAD]      
[PAD]      
[PA

In [ ]:
# Initialize Model
model = BertForABSA(num_labels=2).to(device)

# Compute class weights for imbalance handling
class_labels = np.array([0, 1])
class_weights = compute_class_weight("balanced", classes=class_labels, y=train_df["sentiment"].values)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

print(f"Class Weights: Negative={class_weights[0]:.4f}, Positive={class_weights[1]:.4f}")

# Define weighted loss function
loss_fn = nn.CrossEntropyLoss(weight=class_weights)

# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
num_epochs = 3
num_training_steps = len(train_dataloader) * num_epochs
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=100, #len(train_dataloader) * num_epochs
    num_training_steps=num_training_steps)

# Training Loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}

        loss, logits = model(batch["input_ids"], batch["attention_mask"], batch["aspect_mask"], batch["labels"])
        #loss = loss_fn(logits, batch["labels"])

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}: Loss = {total_loss / len(train_dataloader):.4f}")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Class Weights: Negative=1.5879, Positive=0.7298


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1: Loss = 0.6918
Epoch 2: Loss = 0.6822


In [ ]:
for batch in train_dataloader:
    print("Input IDs:", batch["input_ids"][0])
    print("Aspect Mask:", batch["aspect_mask"][0]) # ✅ Should now contain multiple 1s for aspect words
    print("Labels:", batch["labels"][0])
    break

#***Evaluation***

In [ ]:
# Evaluate Model
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for batch in test_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        _, logits = model(batch["input_ids"], batch["attention_mask"], batch["aspect_mask"])
        predictions = torch.argmax(logits, dim=-1).cpu().numpy()
        y_pred.extend(predictions)
        y_true.extend(batch["labels"].cpu().numpy())

# Print classification report
print(classification_report(y_true, y_pred, target_names=["Negative", "Positive"]))

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc
import seaborn as sns
import matplotlib.pyplot as plt

def evaluate_model(model, dataloader, dataset_name):
    """
    Evaluate model on a given dataset (train/test).
    """
    model.eval()
    y_true, y_pred, y_probs = [], [], []

    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            _, logits = model(batch["input_ids"], batch["attention_mask"], batch["aspect_mask"])

            probs = torch.softmax(logits, dim=-1)[:, 1].cpu().numpy()  # Probability of positive class
            predictions = torch.argmax(logits, dim=-1).cpu().numpy()

            y_pred.extend(predictions)
            y_true.extend(batch["labels"].cpu().numpy())
            y_probs.extend(probs)

    # Compute Metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="binary")
    recall = recall_score(y_true, y_pred, average="binary")
    f1 = f1_score(y_true, y_pred, average="binary")

    print(f"\n📌 {dataset_name} Classification Report:")
    print(classification_report(y_true, y_pred, target_names=["Negative", "Positive"]))

    print(f"\n✅ {dataset_name} Metrics:")
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-score:  {f1:.4f}")

    # Visualization
    plot_confusion_matrix(y_true, y_pred, dataset_name)
    plot_roc_curve(y_true, y_probs, dataset_name)
    plot_precision_recall_curve(y_true, y_probs, dataset_name)

    return y_true, y_pred, y_probs

In [ ]:
def plot_confusion_matrix(y_true, y_pred, dataset_name):
    """
    Plot Confusion Matrix.
    """
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Negative", "Positive"], yticklabels=["Negative", "Positive"])
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix - {dataset_name}")
    plt.show()

In [ ]:
def plot_roc_curve(y_true, y_probs, dataset_name):
    """
    Plot ROC curve and compute AUC score.
    """
    fpr, tpr, _ = roc_curve(y_true, y_probs)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(6, 5))
    plt.plot(fpr, tpr, color="blue", label=f"ROC Curve (AUC = {roc_auc:.2f})")
    plt.plot([0, 1], [0, 1], color="gray", linestyle="--")  # Random baseline
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"ROC Curve - {dataset_name}")
    plt.legend(loc="lower right")
    plt.show()

    print(f"✅ {dataset_name} AUC Score: {roc_auc:.4f}")

In [ ]:
from sklearn.metrics import precision_recall_curve

def plot_precision_recall_curve(y_true, y_probs, dataset_name):
    """
    Plot Precision-Recall Curve.
    """
    precision, recall, _ = precision_recall_curve(y_true, y_probs)

    plt.figure(figsize=(6, 5))
    plt.plot(recall, precision, marker=".", label="Precision-Recall Curve")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title(f"Precision-Recall Curve - {dataset_name}")
    plt.legend()
    plt.show()

In [ ]:
print("\n🔹 Evaluating on Train Set:")
y_train_true, y_train_pred, y_train_probs = evaluate_model(model, train_dataloader, "Train Set")

print("\n🔹 Evaluating on Test Set:")
y_test_true, y_test_pred, y_test_probs = evaluate_model(model, test_dataloader, "Test Set")

#***Hyperparameter Tuning***

In [ ]:
import torch.optim as optim

learning_rates = [5e-5, 3e-5, 2e-5, 1e-5]
best_lr = None
best_val_loss = float("inf")

for lr in learning_rates:
    print(f"\nTraining with Learning Rate: {lr}")

    model = BertForABSA(num_labels=2).to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)

    # Train for 1 epoch and track validation loss
    val_loss = 0
    model.train()

    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        optimizer.zero_grad()
        loss, logits = model(batch["input_ids"], batch["attention_mask"], batch["aspect_mask"], batch["labels"])
        loss.backward()
        optimizer.step()
        val_loss += loss.item()

    avg_val_loss = val_loss / len(train_dataloader)
    print(f"Validation Loss: {avg_val_loss:.4f}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_lr = lr

print(f"\n✅ Best Learning Rate Found: {best_lr}")

# ***Aspect-Based Sentiment Prediction (Inference)***

In [24]:
import torch

# ✅ Define predefined business aspects
PREDEFINED_ASPECTS = ["price", "service", "quality", "delivery", "usability",
                       "item", "product", "performance", "support", "refunds"]

def predict_aspect_sentiment(model, tokenizer, sentence, aspects=PREDEFINED_ASPECTS):
    """
    Predicts sentiment for each predefined aspect in a sentence.
    If an aspect is not present in the sentence, it is marked as "Absent".
    """
    model.eval()
    inputs = tokenizer(sentence, padding="max_length", truncation=True, return_tensors="pt").to(device)

    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]
    sentence_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

    aspect_sentiments = {}

    for aspect in aspects:
        # Tokenize the aspect term
        aspect_tokens = tokenizer.tokenize(aspect.lower())

        # Initialize aspect mask
        aspect_mask = torch.zeros_like(input_ids, dtype=torch.float).to(device)

        # Check if aspect is in the tokenized sentence
        aspect_present = False
        for i in range(len(sentence_tokens) - len(aspect_tokens) + 1):
            if sentence_tokens[i: i + len(aspect_tokens)] == aspect_tokens:
                aspect_present = True
                aspect_mask[0, i: i + len(aspect_tokens)] = 1  # Mark as aspect-related

        # 🚨 Skip aspect if not present in sentence
        if not aspect_present:
            aspect_sentiments[aspect] = "Absent"
            continue

        # Model forward pass
        with torch.no_grad():
            _, logits = model(input_ids, attention_mask, aspect_mask)

        # Get sentiment prediction
        sentiment_label = torch.argmax(logits, dim=-1).cpu().item()
        aspect_sentiments[aspect] = "Positive" if sentiment_label == 1 else "Negative"

    return aspect_sentiments

In [25]:
# ✅ Test Example
sentence = "The food was delicious but the service was terrible."
result = predict_aspect_sentiment(model, tokenizer, sentence)
print(result)

{'food': 'Positive', 'service': 'Positive'}


#***OutPut for Pre-defined Aspects***

In [20]:
import torch

# ✅ Define predefined business aspects
PREDEFINED_ASPECTS = ["price", "service", "quality", "delivery", "usability",
                       "item", "product", "performance", "support", "refunds"]

In [17]:
def debug_aspect_mask(sentence, tokenizer, aspects):
    inputs = tokenizer(sentence, padding="max_length", truncation=True, return_tensors="pt").to(device)

    print("\n📌 Tokenized Sentence:\n")
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    print(tokens)

    for aspect in aspects:
        aspect_tokens = tokenizer.tokenize(aspect.lower())
        aspect_ids = tokenizer.convert_tokens_to_ids(aspect_tokens)

        aspect_mask = torch.zeros_like(inputs["input_ids"], dtype=torch.float).to(device)

        for i in range(len(inputs["input_ids"][0]) - len(aspect_ids) + 1):
            if inputs["input_ids"][0][i: i + len(aspect_ids)].tolist() == aspect_ids:
                aspect_mask[0, i: i + len(aspect_ids)] = 1

        print(f"\n📌 Aspect: {aspect}")
        print("Aspect Mask:", aspect_mask[0].cpu().numpy())  # ✅ Print aspect mask values

# ✅ Run debug test
sentence = "The price is too high, but the service was excellent."
debug_aspect_mask(sentence, tokenizer, PREDEFINED_ASPECTS)


📌 Tokenized Sentence:

['[CLS]', 'the', 'price', 'is', 'too', 'high', ',', 'but', 'the', 'service', 'was', 'excellent', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '

In [21]:
def predict_aspect_sentiment(model, tokenizer, sentence, aspects):
    model.eval()
    inputs = tokenizer(sentence, padding="max_length", truncation=True, return_tensors="pt").to(device)

    aspect_sentiments = {}

    for aspect in aspects:
        aspect_tokens = tokenizer.tokenize(aspect)
        aspect_mask = torch.zeros_like(inputs["input_ids"], dtype=torch.float).to(device)

        for i in range(len(inputs["input_ids"][0]) - len(aspect_tokens) + 1):
            if tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][i: i + len(aspect_tokens)]) == aspect_tokens:
                aspect_mask[0, i: i + len(aspect_tokens)] = 1

        with torch.no_grad():
            _, logits = model(inputs["input_ids"], inputs["attention_mask"], aspect_mask)

        sentiment_label = torch.argmax(logits, dim=-1).cpu().item()
        aspect_sentiments[aspect] = "Positive" if sentiment_label == 1 else "Negative"

    return aspect_sentiments

# Example
sentence = "The food was amazing but the service was terrible."
aspects = ["food", "service"]
result = predict_aspect_sentiment(model, tokenizer, sentence, aspects)
print(result)


#### Test

In [23]:
sentence = "The price is too high, but the service was excellent"
result = predict_aspect_sentiment(model, tokenizer, sentence)

print(result)

{'price': 'Positive', 'service': 'Positive', 'quality': 'Not Found', 'delivery': 'Not Found', 'usability': 'Not Found', 'item': 'Not Found', 'product': 'Not Found', 'performance': 'Not Found', 'support': 'Not Found', 'refunds': 'Not Found'}


#***Output***

In [24]:
import spacy

nlp = spacy.load("en_core_web_sm")

# Define a list of unimportant aspect words to ignore
EXCLUDE_WORDS = {"money", "breath", "thing", "someone", "something", "everything", "anything", "nothing",
                 "one", "stars", "form", "people", "artist", "pop", "credit", "appeal", "half"}

def extract_main_aspects(sentence):
    doc = nlp(sentence)
    aspects = []

    for token in doc:
        if token.pos_ == "NOUN" and (token.dep_ in ["nsubj", "dobj", "pobj"]) and token.text.lower() not in EXCLUDE_WORDS:
            aspects.append(token.text)

    return list(set(aspects))  # Remove duplicates

# Example
sentence = """Dont waste your money. Or hold your breath.: Do they give half stars?
I dont know what the appeal is. She's not pretty and can't sing. I would give her some credit
if she wrote her songs but, like many pop artists, she doesn't. The only halfway decent song in the album is 'Drive Me Crazy,'
but in most, she howls like a dying cat. Go buy Christina Agulara. She has some form of talent."""

print("Extracted Aspects:", extract_main_aspects(sentence))

Extracted Aspects: ['album', 'cat', 'talent', 'songs', 'artists', 'song']


In [23]:
def predict_aspect_sentiment_auto(model, tokenizer, sentence):
    aspects = extract_main_aspects(sentence)
    model.eval()

    # ✅ Tokenize sentence the same way as training
    inputs = tokenizer(sentence, padding="max_length", truncation=True, return_tensors="pt").to(device)

    aspect_sentiments = {}

    for aspect in aspects:
        aspect_tokens = tokenizer.tokenize(aspect)
        aspect_ids = tokenizer.convert_tokens_to_ids(aspect_tokens)

        # ✅ Ensure the aspect_mask highlights only the relevant tokens
        aspect_mask = torch.zeros_like(inputs["input_ids"], dtype=torch.float).to(device)
        for i in range(len(inputs["input_ids"][0]) - len(aspect_ids) + 1):
            if inputs["input_ids"][0][i: i + len(aspect_ids)].tolist() == aspect_ids:
                aspect_mask[0, i: i + len(aspect_ids)] = 1

        with torch.no_grad():
            _, logits = model(inputs["input_ids"], inputs["attention_mask"], aspect_mask)

        sentiment_label = torch.argmax(logits, dim=-1).cpu().item()
        aspect_sentiments[aspect] = "Positive" if sentiment_label == 1 else "Negative"

    return aspect_sentiments


# Example Usage
sentence = """Dont waste your money. Or hold your breath.: Do they give half stars?
I dont know what the appeal is. She's not pretty and can't sing. I would give her some credit
if she wrote her songs but, like many pop artists, she doesn't. The only halfway decent song in the album is 'Drive Me Crazy,'
but in most, she howls like a dying cat. Go buy Christina Agulara. She has some form of talent."""

result = predict_aspect_sentiment_auto(model, tokenizer, sentence)
print(result)

{'song': 'Negative', 'talent': 'Negative', 'artists': 'Negative', 'cat': 'Negative', 'album': 'Negative', 'songs': 'Negative'}


In [25]:
def extract_aspects(sentence):

    doc = nlp(sentence)
    aspects = [token.text for token in doc if token.pos_ in ["NOUN"]]  # Extract only nouns
    return aspects

def predict_aspect_sentiment_auto(model, tokenizer, sentence):

    aspects = extract_aspects(sentence)  # Extract aspects using POS tagging
    model.eval()
    inputs = tokenizer(sentence, padding="max_length", truncation=True, return_tensors="pt").to(device)

    aspect_sentiments = {}

    for aspect in aspects:
        aspect_tokens = tokenizer.tokenize(aspect)
        aspect_mask = torch.zeros_like(inputs["input_ids"], dtype=torch.float).to(device)

        for i in range(len(inputs["input_ids"][0]) - len(aspect_tokens) + 1):
            if tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][i: i + len(aspect_tokens)]) == aspect_tokens:
                aspect_mask[0, i: i + len(aspect_tokens)] = 1

        with torch.no_grad():
            _, logits = model(inputs["input_ids"], inputs["attention_mask"], aspect_mask)

        sentiment_label = torch.argmax(logits, dim=-1).cpu().item()
        aspect_sentiments[aspect] = "Positive" if sentiment_label == 1 else "Negative"

    return aspect_sentiments

# Example
sentence = "Beautiful watch: I was so delighted when my watch arrived. I have been wearing Seiko watches most of my adult life. This one is so beautiful. I love it."

result = predict_aspect_sentiment_auto(model, tokenizer, sentence)
print(result)

{'watch': 'Negative', 'watches': 'Negative', 'adult': 'Negative', 'life': 'Negative', 'one': 'Negative'}


In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer

# Load Pre-trained Model and Tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForABSA(num_labels=2).to(device)  # Ensure model is loaded
model.eval()

# Predefined aspects
PREDEFINED_ASPECTS = ["price", "service", "quality", "food"]

# Function to predict sentiment for a given aspect
def predict_sentiment(model, tokenizer, sentence, aspect):
    inputs = tokenizer(sentence, padding="max_length", truncation=True, return_tensors="pt").to(device)
    aspect_tokens = tokenizer.tokenize(aspect)
    aspect_ids = tokenizer.convert_tokens_to_ids(aspect_tokens)

    aspect_mask = torch.zeros_like(inputs["input_ids"], dtype=torch.float).to(device)
    for i in range(len(inputs["input_ids"][0]) - len(aspect_ids) + 1):
        if inputs["input_ids"][0][i: i + len(aspect_ids)].tolist() == aspect_ids:
            aspect_mask[0, i: i + len(aspect_ids)] = 1

    with torch.no_grad():
        _, logits = model(inputs["input_ids"], inputs["attention_mask"], aspect_mask)
    sentiment_label = torch.argmax(logits, dim=-1).cpu().item()
    return "Positive" if sentiment_label == 1 else "Negative"

# Function to process a CSV file and summarize sentiment
def process_reviews(csv_file):
    df = pd.read_csv(csv_file)
    aspect_sentiment_counts = {aspect: {"Positive": 0, "Negative": 0} for aspect in PREDEFINED_ASPECTS}
    aspect_review_counts = {aspect: 0 for aspect in PREDEFINED_ASPECTS}

    for review in df["review"]:
        for aspect in PREDEFINED_ASPECTS:
            if aspect in review.lower():  # Ensure aspect is in the review
                sentiment = predict_sentiment(model, tokenizer, review, aspect)
                aspect_sentiment_counts[aspect][sentiment] += 1
                aspect_review_counts[aspect] += 1

    # Compute sentiment percentages
    aspect_sentiment_percentages = {}
    for aspect, counts in aspect_sentiment_counts.items():
        total = aspect_review_counts[aspect]
        if total > 0:
            aspect_sentiment_percentages[aspect] = {
                "Negative": round((counts["Negative"] / total) * 100, 2),
                "Positive": round((counts["Positive"] / total) * 100, 2)
            }
        else:
            aspect_sentiment_percentages[aspect] = {"Negative": 0, "Positive": 0}

    return aspect_sentiment_percentages

# Example usage
csv_file = "reviews.csv"  # Replace with your actual file path
result = process_reviews(csv_file)
print(result)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf4 in position 10: invalid continuation byte

#.

Input Representation for Aspect Awareness

In [ ]:
def convert_example_to_features(sentence, aspect, tokenizer, max_length=128):
    """
    Converts a sentence into BERT input features, including an aspect mask.

    - sentence: Full review text
    - aspect: Aspect term (e.g., 'food', 'service')
    - tokenizer: BERT tokenizer
    - max_length: Maximum sequence length
    """
    tokens = tokenizer.tokenize(sentence)
    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # Create attention mask
    attention_mask = [1] * len(input_ids)

    # Generate aspect mask (1 for aspect-related tokens, 0 otherwise)
    aspect_mask = [0] * len(input_ids)
    aspect_tokens = tokenizer.tokenize(aspect)

    for i in range(len(tokens) - len(aspect_tokens) + 1):
        if tokens[i: i + len(aspect_tokens)] == aspect_tokens:
            aspect_mask[i: i + len(aspect_tokens)] = [1] * len(aspect_tokens)

    # Pad sequences
    padding_length = max_length - len(input_ids)
    input_ids += [0] * padding_length
    attention_mask += [0] * padding_length
    aspect_mask += [0] * padding_length

    return {
        "input_ids": torch.tensor(input_ids, dtype=torch.long),
        "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
        "aspect_mask": torch.tensor(aspect_mask, dtype=torch.long)
    }

BERT model for Aspect-Aware Attention

In [ ]:
from transformers import BertPreTrainedModel, BertModel
import torch
import torch.nn as nn

class BertForABSA(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        self.aspect_attention = nn.Linear(config.hidden_size, 1)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, input_ids, attention_mask, aspect_mask, labels=None):
        # Get BERT output
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state  # Shape: (batch_size, seq_len, hidden_dim)

        # Apply aspect-aware attention
        aspect_weights = torch.sigmoid(self.aspect_attention(hidden_states))  # Compute attention scores
        aspect_weights = aspect_weights * aspect_mask.unsqueeze(-1)  # Apply aspect mask
        aspect_representations = hidden_states * aspect_weights  # Scale hidden states by attention weights

        # Pool aspect-aware representations
        pooled_output = aspect_representations.mean(dim=1)

        # Classification layer
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        # Compute loss
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)

        return loss, logits

Training pipeline

In [ ]:
from torch.utils.data import Dataset
import torch

class ABSADataset(Dataset):
    def __init__(self, df, tokenizer, max_length=128):
        self.tokens = df["tokens"].tolist()
        self.aspect_masks = df["aspect_mask"].tolist()
        self.sentiments = df["sentiment"].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        tokens = self.tokens[idx]

        # Ensure tokens is a list
        if isinstance(tokens, str):  # If stored as string, convert to list
            tokens = eval(tokens)
        elif not isinstance(tokens, list):  # If somehow still not list, wrap in list
            tokens = [tokens]

        input_ids = self.tokenizer.convert_tokens_to_ids(tokens)

        # Ensure aspect_mask is a list
        aspect_mask = self.aspect_masks[idx]
        if isinstance(aspect_mask, str):
            aspect_mask = eval(aspect_mask)
        elif not isinstance(aspect_mask, list):
            aspect_mask = [aspect_mask]

        # Create attention mask (1 for actual tokens, 0 for padding)
        attention_mask = [1] * len(input_ids)

        # **🔹 Fix: Ensure consistent sequence length using padding**
        if len(input_ids) > self.max_length:
            input_ids = input_ids[:self.max_length]
            attention_mask = attention_mask[:self.max_length]
            aspect_mask = aspect_mask[:self.max_length]
        else:
            padding_length = self.max_length - len(input_ids)
            input_ids += [0] * padding_length
            attention_mask += [0] * padding_length
            aspect_mask += [0] * padding_length

        return {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
            "aspect_mask": torch.tensor(aspect_mask, dtype=torch.long),
            "labels": torch.tensor(self.sentiments[idx], dtype=torch.long)
        }

print("✅ Dataset class loaded successfully!")


✅ Dataset class loaded successfully!


class imbalance

In [ ]:
import torch
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

train_dataset = ABSADataset(train_df, tokenizer)
test_dataset = ABSADataset(test_df, tokenizer)

# ✅ Fix: Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Fix: Ensure `y` is a NumPy array
class_labels = np.array([0, 1])  # Only two classes now
class_weights = compute_class_weight("balanced", classes=class_labels, y=train_df["sentiment"].values)

class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)

In [ ]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

print("Train and Test DataLoaders are ready!")

Train and Test DataLoaders are ready!


Train

In [ ]:
from transformers import AdamW, get_scheduler
import torch

# Initialize model
model = BertForABSA.from_pretrained("bert-base-uncased", num_labels=2)  # 3 sentiment classes (Neg, Neu, Pos)

# Define optimizer with weight decay for better generalization
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)

num_epochs = 3

# Define learning rate scheduler
num_training_steps = len(train_dataloader) * num_epochs
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=100, num_training_steps=num_training_steps
)

# Set up device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training Loop
print("🚀 Starting Training...")

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    num_batches = len(train_dataloader)

    for step, batch in enumerate(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        loss, logits = model(
            batch["input_ids"],
            batch["attention_mask"],
            batch["aspect_mask"],
            batch["labels"]
        )

        loss = loss_fn(logits, batch["labels"])

        # Backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()

        # Print loss every 50 steps
        if (step + 1) % 50 == 0 or (step + 1) == num_batches:
            print(f"Epoch [{epoch+1}/{num_epochs}] Step [{step+1}/{num_batches}] - Loss: {loss.item():.4f}")

    avg_loss = total_loss / num_batches
    print(f"✅ Epoch {epoch+1} Completed! Avg Loss: {avg_loss:.4f}")

print("🎉 Training Finished!")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForABSA were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['aspect_attention.bias', 'aspect_attention.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


🚀 Starting Training...
Epoch [1/3] Step [50/66] - Loss: 0.6844
Epoch [1/3] Step [66/66] - Loss: 0.7161
✅ Epoch 1 Completed! Avg Loss: 0.6916
Epoch [2/3] Step [50/66] - Loss: 0.7050
Epoch [2/3] Step [66/66] - Loss: 0.6266
✅ Epoch 2 Completed! Avg Loss: 0.6270
Epoch [3/3] Step [50/66] - Loss: 0.4958
Epoch [3/3] Step [66/66] - Loss: 0.4560
✅ Epoch 3 Completed! Avg Loss: 0.4771
🎉 Training Finished!


Evaluate

In [ ]:
from sklearn.metrics import classification_report

def evaluate_model(model, dataloader):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            _, logits = model(batch["input_ids"], batch["attention_mask"], batch["aspect_mask"])
            predictions = torch.argmax(logits, dim=-1).cpu().numpy()
            y_pred.extend(predictions)
            y_true.extend(batch["labels"].cpu().numpy())

    print(classification_report(y_true, y_pred, target_names=["Negative", "Positive"]))

# Run evaluation
evaluate_model(model, test_dataloader)


              precision    recall  f1-score   support

    Negative       0.82      0.66      0.73        82
    Positive       0.86      0.93      0.89       180

    accuracy                           0.85       262
   macro avg       0.84      0.80      0.81       262
weighted avg       0.84      0.85      0.84       262



hyperparameter tunning

In [ ]:
import torch
import torch.optim as optim
from transformers import get_scheduler
from sklearn.model_selection import ParameterGrid
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report

# Define possible hyperparameters
param_grid = {
    "learning_rate": [2e-5, 3e-5, 5e-5],  # Common BERT fine-tuning rates
    "batch_size": [8, 16],  # Trade-off between speed and performance
    "weight_decay": [0.01, 0.001]  # Regularization to avoid overfitting
}

# Create all possible hyperparameter combinations
param_combinations = list(ParameterGrid(param_grid))

best_model = None
best_f1_score = 0  # Track the best model based on F1-score

for params in param_combinations:
    print(f"\n🚀 Training with params: {params}")

    # Load model
    model = BertForABSA.from_pretrained("bert-base-uncased", num_labels=2)
    model.to(device)

    # Optimizer & Learning Rate Scheduler
    optimizer = optim.AdamW(model.parameters(), lr=params["learning_rate"], weight_decay=params["weight_decay"])
    num_training_steps = len(train_dataloader) * 3  # Assuming 3 epochs
    lr_scheduler = get_scheduler(
        "linear", optimizer=optimizer, num_warmup_steps=100, num_training_steps=num_training_steps
    )

    # Loss function
    criterion = torch.nn.CrossEntropyLoss()

    # Data Loaders
    train_dataloader = DataLoader(train_dataset, batch_size=params["batch_size"], shuffle=True)
    val_dataloader = DataLoader(test_dataset, batch_size=params["batch_size"])

    # Training Loop
    for epoch in range(3):
        model.train()
        total_loss = 0
        for batch in train_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            optimizer.zero_grad()

            loss, logits = model(batch["input_ids"], batch["attention_mask"], batch["aspect_mask"], batch["labels"])
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            lr_scheduler.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1} - Loss: {total_loss / len(train_dataloader)}")

    # Validate Model
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for batch in val_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            _, logits = model(batch["input_ids"], batch["attention_mask"], batch["aspect_mask"])
            predictions = torch.argmax(logits, dim=-1).cpu().numpy()
            y_pred.extend(predictions)
            y_true.extend(batch["labels"].cpu().numpy())

    # Compute F1-score
    report = classification_report(y_true, y_pred, target_names=["Negative", "Positive"], output_dict=True)
    f1_score = report["macro avg"]["f1-score"]
    print(classification_report(y_true, y_pred, target_names=["Negative", "Positive"]))

    # Save best model based on F1-score
    if f1_score > best_f1_score:
        best_f1_score = f1_score
        best_model = model
        best_params = params

# Save the best model
best_model.save_pretrained("best_absa_model")
print(f"✅ Best model saved with params: {best_params} (F1-score: {best_f1_score:.4f})")


🚀 Training with params: {'batch_size': 8, 'learning_rate': 2e-05, 'weight_decay': 0.01}


Some weights of BertForABSA were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['aspect_attention.bias', 'aspect_attention.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 - Loss: 0.6534200262932377
Epoch 2 - Loss: 0.5424236830409247
Epoch 3 - Loss: 0.5091745596350604
              precision    recall  f1-score   support

    Negative       0.70      0.39      0.50        82
    Positive       0.77      0.92      0.84       180

    accuracy                           0.76       262
   macro avg       0.73      0.66      0.67       262
weighted avg       0.75      0.76      0.73       262


🚀 Training with params: {'batch_size': 8, 'learning_rate': 2e-05, 'weight_decay': 0.001}


Some weights of BertForABSA were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['aspect_attention.bias', 'aspect_attention.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 - Loss: 0.6641075499640167
Epoch 2 - Loss: 0.5481806626074187
Epoch 3 - Loss: 0.395243507873921
              precision    recall  f1-score   support

    Negative       0.80      0.65      0.72        82
    Positive       0.85      0.93      0.89       180

    accuracy                           0.84       262
   macro avg       0.83      0.79      0.80       262
weighted avg       0.84      0.84      0.83       262


🚀 Training with params: {'batch_size': 8, 'learning_rate': 3e-05, 'weight_decay': 0.01}


Some weights of BertForABSA were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['aspect_attention.bias', 'aspect_attention.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
